Address component extraction from the text.

PS: Please select GPU as a runtime for fater processing.

Owner: Chetan Khadke khadkechetan@gmail.com

In [1]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pandas as pd
import re, math
import numpy as np
import pandas as pd
from flair.data import Corpus
from flair.datasets import ColumnCorpus

In [3]:
from flair.embeddings import FlairEmbeddings, TransformerWordEmbeddings
from flair.datasets import CONLL_03
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.embeddings import StackedEmbeddings

In [4]:
columns = ['Building_Name', 'Building_Number', 'City', 'Recipient',
       'Street_Name', 'Zip_Code', 'State', 'Country']

In [5]:
count=0
labels = set()
def generate_data(file_name):
    data = pd.read_csv(file_name)
    def get_value(x):
        global count
        results = []
        address = x["Address"]
        previous = ""
        _temp = x.to_dict()
        for each in address.split():
            match = False
            for col in columns:
                if each.strip(" ,.-") in str(_temp[col]):
                    if previous == col:
                        results.append((count, each, "I-"+col))
                        labels.add("I-"+col)
                    else:
                        results.append((count, each, "B-"+col)) 
                        labels.add("B-"+col)
                    previous = col
                    match = True
                    break
            if not match:
                results.append((count, each, "O")) 
                labels.add("O")
                match = False
        results.append(("", "",""))
        count += 1
        return results
    values = data[:].apply(lambda x:get_value(x), axis=1)
    flat_list = [item for sublist in values for item in sublist]
    train_df = pd.DataFrame(flat_list, columns=["sentence_id", "words", "labels"])
    return train_df

In [9]:
# create a train folder
os.makedirs("train/", exist_ok = True) 

In [10]:
train_df = generate_data("us-train-dataset.csv")

In [11]:
test_df = generate_data("us-test-dataset.csv")

In [12]:
train_df[["words", "labels"]].tail(20)

,words,labels
3169,4538,B-Street_Name
3170,Us,I-Street_Name
3171,Hwy,I-Street_Name
3172,231,I-Street_Name
3173,36092,B-Zip_Code
3174,Wetumpka,B-City
3175,AL,B-State
3176,United,B-Country
3177,States,I-Country
3178,,


In [13]:
train_df[["words", "labels"]].to_csv(r'train/train.txt', header=None, index=None, sep=' ')

In [14]:
test_df[["words", "labels"]].to_csv(r'train/test.txt', header=None, index=None, sep=' ')

In [15]:
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = 'train/'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'test.txt')

2023-01-27 17:05:34,287 Reading data from train
2023-01-27 17:05:34,289 Train: train/train.txt
2023-01-27 17:05:34,290 Dev: train/test.txt
2023-01-27 17:05:34,291 Test: train/test.txt


In [16]:
print(len(corpus.train))

354


In [17]:
# tag to predict
columns = {0: 'text', 1: 'ner'}
tag_type = columns[1]
# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary("ner")

2023-01-27 17:05:37,044 Computing label dictionary. Progress:


354it [00:00, 35328.44it/s]

2023-01-27 17:05:37,067 Dictionary created for label 'ner' with 9 values: Street_Name (seen 353 times), City (seen 287 times), State (seen 265 times), Country (seen 265 times), Zip_Code (seen 255 times), Building_Number (seen 104 times), Building_Name (seen 20 times), Recipient (seen 18 times)


In [18]:
tag_dictionary

In [19]:


# init Flair embeddings
flair_forward_embedding = FlairEmbeddings('multi-forward')
flair_backward_embedding = FlairEmbeddings('multi-backward')

# init multilingual BERT
bert_embedding = TransformerWordEmbeddings('bert-base-multilingual-cased')

# now create the StackedEmbedding object that combines all embeddings
embeddings = StackedEmbeddings(
    embeddings=[flair_forward_embedding, flair_backward_embedding, bert_embedding])

In [22]:

# 2. what label do we want to predict?
label_type = 'ner'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# 4. initialize embedding stack with Flair and GloVe
embedding_types = [
    # WordEmbeddings('glove'),
    FlairEmbeddings('news-forward'),
    FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/sota-ner-flair',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=50)

2023-01-27 17:09:32,380 Computing label dictionary. Progress:


354it [00:00, 17891.11it/s]

2023-01-27 17:09:32,411 Dictionary created for label 'ner' with 9 values: Street_Name (seen 353 times), City (seen 287 times), State (seen 265 times), Country (seen 265 times), Zip_Code (seen 255 times), Building_Number (seen 104 times), Building_Name (seen 20 times), Recipient (seen 18 times)
Dictionary with 9 tags: <unk>, Street_Name, City, State, Country, Zip_Code, Building_Number, Building_Name, Recipient


2023-01-27 17:09:32,771 SequenceTagger predicts: Dictionary with 33 tags: O, S-Street_Name, B-Street_Name, E-Street_Name, I-Street_Name, S-City, B-City, E-City, I-City, S-State, B-State, E-State, I-State, S-Country, B-Country, E-Country, I-Country, S-Zip_Code, B-Zip_Code, E-Zip_Code, I-Zip_Code, S-Building_Number, B-Building_Number, E-Building_Number, I-Building_Number, S-Building_Name, B-Building_Name, E-Building_Name, I-Building_Name, S-Recipient, B-Recipient, E-Recipient, I-Recipient
2023-01-27 17:09:32,995 ----------------------------------------------------------------------------------------------------
2023-01-27 17:09:33,000 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_1): Flai

/usr/local/lib/python3.8/dist-packages/flair/trainers/trainer.py:64: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


...

2023-01-27 17:10:09,350 Evaluating as a multi-label problem: False
2023-01-27 17:10:09,362 DEV : loss 0.29312923550605774 - f1-score (micro avg)  0.8706
2023-01-27 17:10:09,368 BAD EPOCHS (no improvement): 0
2023-01-27 17:10:09,371 saving best model


2023-01-27 17:10:11,130 ----------------------------------------------------------------------------------------------------
2023-01-27 17:10:11,132 loading file resources/taggers/sota-ner-flair/best-model.pt
2023-01-27 17:10:11,634 SequenceTagger predicts: Dictionary with 35 tags: O, S-Street_Name, B-Street_Name, E-Street_Name, I-Street_Name, S-City, B-City, E-City, I-City, S-State, B-State, E-State, I-State, S-Country, B-Country, E-Country, I-Country, S-Zip_Code, B-Zip_Code, E-Zip_Code, I-Zip_Code, S-Building_Number, B-Building_Number, E-Building_Number, I-Building_Number, S-Building_Name, B-Building_Name, E-Building_Name, I-Building_Name, S-Recipient, B-Recipient, E-Recipient, I-Recipient, <START>, <STOP>


100%|██████████| 1/1 [00:00<00:00,  7.13it/s]

2023-01-27 17:10:12,035 Evaluating as a multi-label problem: False
2023-01-27 17:10:12,054 0.8706	0.8706	0.8706	0.7789
2023-01-27 17:10:12,059 
Results:
- F-score (micro) 0.8706
- F-score (macro) 0.7953
- Accuracy 0.7789

By class:
                 precision    recall  f1-score   support

    Street_Name     0.7895    0.8333    0.8108        18
Building_Number     0.9412    1.0000    0.9697        16
           City     0.8333    0.9091    0.8696        11
          State     1.0000    1.0000    1.0000        11
        Country     1.0000    1.0000    1.0000        11
       Zip_Code     1.0000    0.7000    0.8235        10
      Recipient     0.8000    1.0000    0.8889         4
  Building_Name     0.0000    0.0000    0.0000         4

      micro avg     0.8706    0.8706    0.8706        85
      macro avg     0.7955    0.8053    0.7953        85
   weighted avg     0.8663    0.8706    0.8643        85

2023-01-27 17:10:12,061 ---------------------------------------------------------

{'test_score': 0.8705882352941176,
 'dev_score_history': [0.1271676300578035,
  0.24864864864864863,
  0.4161849710982659,
  0.6749999999999999,
  0.6878980891719745,
  0.6735751295336788,
  0.6918238993710691,
  0.8484848484848485,
  0.8342857142857144,
  0.7749999999999999,
  0.8248587570621468,
  0.7764705882352941,
  0.8275862068965517,
  0.8620689655172413,
  0.8314606741573034,
  0.813953488372093,
  0.8284023668639053,
  0.8372093023255814,
  0.8421052631578947,
  0.8433734939759034,
  0.8554216867469879,
  0.8470588235294118,
  0.8433734939759034,
  0.8421052631578947,
  0.8452380952380952,
  0.8554216867469879,
  0.8433734939759034,
  0.8622754491017964,
  0.8433734939759034,
  0.8654970760233918,
  0.863905325443787,
  0.8622754491017964,
  0.8654970760233918,
  0.8433734939759034,
  0.8622754491017964,
  0.8622754491017964,
  0.8622754491017964,
  0.8622754491017964,
  0.8622754491017964,
  0.863905325443787,
  0.863905325443787,
  0.863905325443787,
  0.8622754491017964,
  

# Transformer Embeddings 

Transformer based embedding. 

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='xlm-roberta-large',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type='ner',
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. run fine-tuning
trainer.fine_tune('resources/taggers/sota-ner-flert',
                  learning_rate=5.0e-6,
                  mini_batch_size=4,
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  )

In [ ]:

from flair.embeddings import WordEmbeddings, StackedEmbeddings, TokenEmbeddings
from typing import List
embedding_types : List[TokenEmbeddings] = [
        WordEmbeddings('glove'),
        ## other embeddings
        ]
embeddings : StackedEmbeddings = StackedEmbeddings(
                                 embeddings=embedding_types)

from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

In [21]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load('/content/resources/taggers/sota-ner-flair/best-model.pt')
# create example sentence
sentence = Sentence('My address is 223 NW STATE STRE, DOVER, DE 19001, United States.')
# predict the tags
model.predict(sentence)
# print(sentence.to_tagged_string())
print(sentence.labels)
# ignore the lower confidence > 0.6, Train with more sample to reduce the false positive

2023-01-27 17:07:57,315 loading file /content/resources/taggers/sota-ner-flair/best-model.pt
2023-01-27 17:07:57,811 SequenceTagger predicts: Dictionary with 35 tags: O, S-Street_Name, B-Street_Name, E-Street_Name, I-Street_Name, S-City, B-City, E-City, I-City, S-State, B-State, E-State, I-State, S-Country, B-Country, E-Country, I-Country, S-Zip_Code, B-Zip_Code, E-Zip_Code, I-Zip_Code, S-Building_Number, B-Building_Number, E-Building_Number, I-Building_Number, S-Building_Name, B-Building_Name, E-Building_Name, I-Building_Name, S-Recipient, B-Recipient, E-Recipient, I-Recipient, <START>, <STOP>
['Span[0:1]: "My"'/'Building_Name' (0.5949), 'Span[1:3]: "address is"'/'Building_Name' (0.461), 'Span[3:4]: "223"'/'Building_Number' (0.9896), 'Span[4:8]: "NW STATE STRE ,"'/'Street_Name' (0.8287), 'Span[8:10]: "DOVER ,"'/'City' (0.5568), 'Span[10:11]: "DE"'/'State' (0.8656), 'Span[11:12]: "19001"'/'Zip_Code' (0.9248), 'Span[12:13]: ","'/'State' (0.3515), 'Span[13:16]: "United States ."'/'Countr